In [9]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath('getters/'))
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [10]:
import pandas as pd
from getters.get_data import GetExpectedInflation
import getters.urls_dict as u

### Carregando dados da inflação esperada através de scraping

In [11]:
    url = u.urls.get('Inflação Esperada')
    headers = u.headers
    get_exp = GetExpectedInflation(url, headers)

    headings, ano, mes, value = get_exp.run()

Scraping url: http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=1693254712&module=M
Approved access


In [12]:
exp = pd.DataFrame(data={'ano':ano,'mes':mes,'valor':value})
exp.rename(columns={'valor':headings[0].split('-')[0]}, inplace=True)

In [14]:
exp.head(15)

,ano,mes,Expectativa média de Inflação
0,2001,07,"6,40"
1,2001,08,"6,57"
2,2001,09,"6,53"
3,2001,10,"6,49"
4,2001,11,"5,57"
5,2001,12,"4,84"
6,2002,01,"4,76"
7,2002,02,"4,70"
8,2002,03,"4,94"
9,2002,04,"4,90"


### Carregando dados do IPCA obtidos através de scraping

In [15]:
FILE = 'IPCA-serie-historica.csv/mes_brasil.csv'

In [16]:
ipca = pd.read_csv('../getters/Data/extracted/'+FILE)

In [17]:
ipca.head()

,ano,mes,indice,variacao_mensal,variacao_trimestral,variacao_semestral,variacao_anual,variacao_doze_meses
0,1979,12,7.618300e-09,NaN,NaN,NaN,NaN,NaN
1,1980,1,8.122300e-09,6.62,NaN,NaN,6.62,NaN
2,1980,2,8.497300e-09,4.62,NaN,NaN,11.54,NaN
3,1980,3,9.010400e-09,6.04,18.27,NaN,18.27,NaN
4,1980,4,9.486700e-09,5.29,16.80,NaN,24.53,NaN


### Carregando dados da taxa de juros básica via scraping

### Iniciando tratamento dos dados

In [31]:
ipca = ipca[['ano','mes','indice','variacao_mensal']]

In [32]:
exp.head()

,ano,mes,Expectativa média de Inflação
0,2001,7,"6,40"
1,2001,8,"6,57"
2,2001,9,"6,53"
3,2001,10,"6,49"
4,2001,11,"5,57"


In [33]:
exp['ano'] = exp['ano'].astype(int)
exp['mes'] = exp['mes'].astype(int)

In [34]:
ipca_exp = ipca.merge(exp, on=['ano','mes'])

In [40]:
ipca_exp.head()

,ano,mes,indice,variacao_mensal,Expectativa média de Inflação
0,2001,7,1756.28,1.33,"6,40"
1,2001,8,1768.57,0.70,"6,57"
2,2001,9,1773.52,0.28,"6,53"
3,2001,10,1788.24,0.83,"6,49"
4,2001,11,1800.94,0.71,"5,57"


In [41]:
from xgboost.sklearn import XGBRegressor

ModuleNotFoundError: No module named 'xgboost'